In [39]:
from pyspark.sql import SparkSession


In [40]:
spark = SparkSession.builder.appName("interview").getOrCreate()

In [41]:
spark

### 1. creae empty rdd and dataframe

In [14]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

# create empty rdd without schema
rdd_wthout_schema = spark.sparkContext.emptyRDD()
rdd_wthout_schema.collect()

# another way to creae rdd 
rdd1 = spark.sparkContext.parallelize([])

# create empty rdd with schema 
#rdd_empty_schema = 

# create empty dataframe with schema 
schema = StructType(
    [
        StructField('id',IntegerType(),True),
        StructField("name",StringType(),True),
        StructField("email",StringType(),True)
    ]
)


# create empty dataframe 
df = rdd_wthout_schema.toDF(schema)
df.show()
# create empty dataframe without schema 
from pyspark.sql.types import  StructType
emptydf_without_schema = spark.createDataFrame([],StructType([]))
emptydf_without_schema.show()

## create empty dataframe with schema 
empty_df_withschema = spark.createDataFrame([],schema=schema)


In [ ]:
### 2. script of question

In [20]:
# 1. merging the tuple using scala/python
tuple_list = [(10,15),(12,20),(18,28),(30,40),(40,50),(55,60)]

add_tuple = ()
for i in tuple_list:
    add_tuple+=i 

add_tuple


(10, 15, 12, 20, 18, 28, 30, 40, 40, 50, 55, 60)

In [21]:
type(add_tuple)

tuple

In [22]:
# 
list1 = [1,2,3,4,5]
list1 = [1,2,3,4,5]

In [48]:
# Q. input text file holding single row eith pipe delimited as shown below. how 
# will you apply break to every 5th occurrence of pip delimiter and display as show below.

from pyspark.sql.functions import regexp_replace,explode,split
# read file 
df1 = spark.read.csv("pipe.txt")
df1.show(truncate=False)

# replace evry 5th | with -
df2 = df1.withColumn("chk",regexp_replace("_c0","(.*?\\|){5}","$0-"))

# explode after spliting to get each rows 
df3 = df2.withColumn("col_expl",explode(split("chk","\|-"))).select("col_expl")

# schema 
col = ["name","Field","exp","tech","mob"]
# split on | and give schema 
df4 = df3.rdd.map(lambda x: x[0].split("|")).toDF(schema=col)
df4.show()


+-----------------------------------------------------------------------------------------------------------------------+
|_c0                                                                                                                    |
+-----------------------------------------------------------------------------------------------------------------------+
|Azar|BE|8|BigData|9273564321|Ramesh|BTech|3|Java|8685936472|Partiban|ME|6|dotNet|8756432542|Magesh|MCA|8|DBA|8765435242|
+-----------------------------------------------------------------------------------------------------------------------+

+--------+-----+---+-------+----------+
|    name|Field|exp|   tech|       mob|
+--------+-----+---+-------+----------+
|    Azar|   BE|  8|BigData|9273564321|
|  Ramesh|BTech|  3|   Java|8685936472|
|Partiban|   ME|  6| dotNet|8756432542|
|  Magesh|  MCA|  8|    DBA|8765435242|
+--------+-----+---+-------+----------+



In [49]:
spark.stop()

In [22]:
# photon intereveiw question
#1. read jon file data_photo 
# output -> raj|Mumbai|python raj|mubai|scala raj|mumbai|magodb vijay|mysure|spark vijay|mysure|ter
df = spark.read.format('json').load('data_photo.json')
from pyspark.sql.functions import  explode
df.withColumn("skill1",explode("skill")).select("name","location","skill1").show()
# ========================================================================================================




#2. read log file and get error log in separtate file error_runlog.txt using python 
# pysaprk
text = spark.read.text("log.txt")
error = text.filter(text.value.contains("Error"))
error.write.mode("overwrite").text("error_runlog.txt")

# python 


with open("log.txt") as f:
    lines = f.readlines()
  
with open("errorlog.txt","w") as e:
    for row in lines:
        if "Error" in row:
            e.write(row)    

# =======================================================================================

# 3. find the second letest login tiem for every user
login = spark.read.csv("login.csv",inferSchema=True,header=True)
# filter login data
log_filter = login.filter(login.act_type=="Login")
# convert to timestamp 
log_time = log_f.withColumn("time",to_timestamp(unix_timestamp(log_f.act_ts,"yyyy-MM-dd hh:mm a")))

from pyspark.sql.window import Window
from  pyspark.sql.functions import rank,col 
# partition and order by 
windowPartition = Window.partitionBy("user_id").orderBy(col("time").desc())
final = log_time.withColumn("rank",rank().over(windowPartition))

# final output 
fi = final.filter(final.rank==2).select("user_id","act_type","act_ts")
fi.show()

+-----+--------+--------+
| name|location|  skill1|
+-----+--------+--------+
|  raj|  mumbai|  python|
|  raj|  mumbai|   scala|
|  raj|  mumbai| mangodb|
|vijay|  mysure|teradata|
|vijay|  mysure|   spark|
+-----+--------+--------+



DataFrame[user_id: int, act_type: string, act_ts: string]

In [66]:
login = spark.read.csv("login.csv",inferSchema=True,header=True)
login

DataFrame[user_id: int, act_type: string, act_ts: string]

In [6]:

login.show()

+-------+--------+-------------------+
|user_id|act_type|             act_ts|
+-------+--------+-------------------+
|    101|   Login| 2022-05-23 9:00 AM|
|    101|  Logout| 2022-05-23 9:30 AM|
|    101|   Login|2022-05-23 10:00 AM|
|    101|  Logout|2022-05-23 10:30 AM|
|    102|   Login| 2022-05-23 9:00 AM|
|    102|  Logout| 2022-05-23 9:30 AM|
|    102|   Login|2022-05-23 10:00 AM|
|    102|  Logout|2022-05-23 10:30 AM|
+-------+--------+-------------------+



In [67]:
# from pyspark.sql.functions import to_timestamp
# df=spark.read.csv(fp,header=True)
# df=df.withColumn('time',to_timestamp("Date","MM/dd/yyyy hh:mm:ss a"))
# df.select("Case Number",'time','Date').show(5,False)

#spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
#login.withColumn("dat",to_timestamp("act_ts","yyyy-MM-dd hh:mm a")).show()


import pyspark.sql.functions as f
log_f = login.filter(login.act_type=="Login")



log_time = log_f.withColumn("time",unix_timestamp(log_f.act_ts,"yyyy-MM-dd hh:mm a"))
log_time.sort("time").show()
from pyspark.sql.window import Window
from  pyspark.sql.functions import rank,col 
windowPartition = Window.partitionBy("user_id").orderBy(col("time").desc())
final = log_time.withColumn("rank",rank().over(windowPartition))
fi = final.filter(final.rank==2).select("user_id","act_type","act_ts")
fi.show()

In [24]:
df1=spark.createDataFrame(
        data = [ ("1","2019-06-24 12:01:19.000")],
        schema=["id","input_timestamp"])
df1.printSchema()

root
 |-- id: string (nullable = true)
 |-- input_timestamp: string (nullable = true)



In [25]:
from pyspark.sql.functions import to_timestamp, to_date, date_format

df2 = df1.withColumn("time",to_timestamp("input_timestamp"))

In [28]:
df2.show()

+---+--------------------+-------------------+
| id|     input_timestamp|               tiem|
+---+--------------------+-------------------+
|  1|2019-06-24 12:01:...|2019-06-24 12:01:19|
+---+--------------------+-------------------+



In [27]:
df1.select(to_date(df1.input_timestamp,'MM-dd-yyyy HH:mm:ss.SSSS')) \
  .show()

+------------------------------------------------------+
|to_date(`input_timestamp`, 'MM-dd-yyyy HH:mm:ss.SSSS')|
+------------------------------------------------------+
|                                                  null|
+------------------------------------------------------+



In [36]:
df2.show()

+---+--------------------+-------------------+
| id|     input_timestamp|               tiem|
+---+--------------------+-------------------+
|  1|2019-06-24 12:01:...|2019-06-24 12:01:19|
+---+--------------------+-------------------+



In [ ]:

from pyspark.sql import SparkSession

# Create SparkSession

df=spark.createDataFrame(
        data = [ ("1","2019-06-24 12:01:19.000")],
        schema=["id","input_timestamp"])
df.printSchema()


from pyspark.sql.functions import *

# Using Cast to convert Timestamp String to DateType
df.withColumn('date_type', col('input_timestamp').cast('date')) \
       .show(truncate=False)

# Using Cast to convert TimestampType to DateType
df.withColumn('date_type', to_timestamp('input_timestamp').cast('date')) \
  .show(truncate=False)

df.select(to_date(lit('06-24-2019 12:01:19.000'),'MM-dd-yyyy HH:mm:ss.SSSS')) \
  .show()
  
#Timestamp String to DateType
df.withColumn("date_type",to_date("input_timestamp")) \
  .show(truncate=False)

#Timestamp Type to DateType
df.withColumn("date_type",to_date(current_timestamp())) \
  .show(truncate=False) 

df.withColumn("ts",to_timestamp(col("input_timestamp"))) \
  .withColumn("datetype",to_date(col("ts"))) \
  .show(truncate=False)
    
#SQL TimestampType to DateType
spark.sql("select to_date(current_timestamp) as date_type")
#SQL CAST TimestampType to DateType
spark.sql("select date(to_timestamp('2019-06-24 12:01:19.000')) as date_type")
#SQL CAST timestamp string to DateType
spark.sql("select date('2019-06-24 12:01:19.000') as date_type")
#SQL Timestamp String (default format) to DateType
spark.sql("select to_date('2019-06-24 12:01:19.000') as date_type")
#SQL Custom Timeformat to DateType
spark.sql("select to_date('06-24-2019 12:01:19.000','MM-dd-yyyy HH:mm:ss.SSSS') as date_type")


In [29]:
spark.stop()

In [57]:
df=spark.createDataFrame(
        data = [ ("1","2019-06-24 12:01:19"),("2","2019-09-21 10:38:00"),("3","2019-05-21 10:20:00")],
        schema=["id","in_time"])

# ("1","2019-06-24 12:01:19 AM"),("2","2019-09-21 10:38:00 PM)"

df.printSchema()
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
from pyspark.sql.functions import  to_date,to_timestamp,col,unix_timestamp
df1 =df.withColumn("time",to_timestamp(unix_timestamp(df.in_time,"yyyy-MM-dd hh:mm:ss")))
df.printSchema()
df1.sort("time").show()

root
 |-- id: string (nullable = true)
 |-- in_time: string (nullable = true)



In [64]:
df=spark.createDataFrame(
        data = [ ("1","2019-06-24 12:01 AM"),("2","2019-09-21 10:38 AM"),("3","2019-05-21 10:20 PM"),("4","2019-05-21 10:20 AM")],
        schema=["id","in_time"])

In [65]:
df.withColumn("time",to_timestamp(unix_timestamp(df.in_time,"yyyy-MM-dd hh:mm a"))).show()

+---+-------------------+-------------------+
| id|            in_time|               time|
+---+-------------------+-------------------+
|  1|2019-06-24 12:01 AM|2019-06-24 00:01:00|
|  2|2019-09-21 10:38 AM|2019-09-21 10:38:00|
|  3|2019-05-21 10:20 PM|2019-05-21 22:20:00|
|  4|2019-05-21 10:20 AM|2019-05-21 10:20:00|
+---+-------------------+-------------------+



In [36]:
df1.productname

0         ortho
1      robotics
2         angle
3    pyrosphere
Name: productname, dtype: object

In [37]:
# compare two csv file 

with open('emp.csv', 'r') as t1, open('emp2.csv', 'r') as t2:
    fileone = t1.readlines()
    filetwo = t2.readlines()


with open('update.csv', 'w') as outFile:
    for line in filetwo:
        if line not in fileone:
            outFile.write(line)


